## Preprocessing

In [34]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [35]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [36]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [37]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [38]:
# Choose a cutoff value (e.g., 1000) for considering categories as "Other"
cutoff_value = 500

categories_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()
application_df['APPLICATION_TYPE'].replace(categories_to_replace, 'Other', inplace=True)
print(application_df['APPLICATION_TYPE'].value_counts())


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [39]:
# Look at the value counts of 'CLASSIFICATION' column
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [40]:
# Filter the value counts of 'CLASSIFICATION' column for categories with counts greater than 1
filtered_classification_counts = classification_counts[classification_counts > 1]
print(filtered_classification_counts)


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [41]:
# Determine the cutoff value
classification_counts = application_df['CLASSIFICATION'].value_counts()
cutoff_value = 1000

# Create a list of classifications to be replaced with "Other"
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace less frequent classifications with "Other" in the 'CLASSIFICATION' column
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")
print(application_df['CLASSIFICATION'].value_counts())


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [42]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)
print(application_df_encoded.head())


   STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                   False   
1       1   108590              1                   False   
2       1     5000              0                   False   
3       1     6692              1                   False   
4       1   142590              1                   False   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                  True                 False                False   
1                 False                 False                 True   
2                 False                 False                False   
3                 False                 False                 True   
4                 False                 False                 True   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                False                False                False  ...   
1                False                False                False  

In [43]:
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)  # Assuming 'IS_SUCCESSFUL' is your target column
y = application_df_encoded['IS_SUCCESSFUL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (27439, 43)
X_test shape: (6860, 43)
y_train shape: (27439,)
y_test shape: (6860,)


In [44]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [45]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
import tensorflow as tf

input_shape = X_train.shape[1]  # Example: X_train.shape[1] if X_train contains your features

# Define the model
nn = tf.keras.models.Sequential()

#first hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=input_shape))
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 128)               5632      
                                                                 
Total params: 5632 (22.00 KB)
Trainable params: 5632 (22.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [47]:
import numpy as np
# Convert data type
X_train = np.array(X_train, dtype=np.float32)

# Convert data type
y_train = np.array(y_train, dtype=np.int32)

# Train the model
history = nn.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
686/686 [==============================] - 2s 3ms/step - loss: 7.6624 - accuracy: 0.0000e+00 - val_loss: 7.6653 - val_accuracy: 0.0000e+00
Epoch 2/10
686/686 [==============================] - 2s 4ms/step - loss: 7.6624 - accuracy: 0.0000e+00 - val_loss: 7.6653 - val_accuracy: 0.0000e+00
Epoch 3/10
686/686 [==============================] - 2s 3ms/step - loss: 7.6624 - accuracy: 0.0000e+00 - val_loss: 7.6653 - val_accuracy: 0.0000e+00
Epoch 4/10
686/686 [==============================] - 2s 2ms/step - loss: 7.6624 - accuracy: 0.0000e+00 - val_loss: 7.6653 - val_accuracy: 0.0000e+00
Epoch 5/10
686/686 [==============================] - 2s 2ms/step - loss: 7.6624 - accuracy: 0.0000e+00 - val_loss: 7.6653 - val_accuracy: 0.0000e+00
Epoch 6/10
686/686 [==============================] - 2s 2ms/step - loss: 7.6624 - accuracy: 0.0000e+00 - val_loss: 7.6653 - val_accuracy: 0.0000e+00
Epoch 7/10
686/686 [==============================] - 2s 2ms/step - loss: 7.6624 - accuracy: 0.0000e

In [48]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 4.8753 - accuracy: 0.0010 - 370ms/epoch - 2ms/step
Loss: 4.875323295593262, Accuracy: 0.0010204081190750003


In [49]:
# Export our model to HDF5 file
model_path = 'model.h5'

# Save the model to HDF5 format
nn.save(model_path)

print(f"Model saved to {model_path}")

Model saved to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
